In [55]:
import requests
import json
from datetime import datetime
import pandas as pd
from nacl.bindings import crypto_sign

import time 

rootApiUrl = "https://api.dmarket.com"
public_key = '3da7a04c31f0a5305370ee0dd2fa51a6cd338268efa687c1f424784a8d829723'
secret_key = 'dfe8772a7374397c00dab6e5e3ad37939ba71f64ac1889a905c95df5ff6cd62f3da7a04c31f0a5305370ee0dd2fa51a6cd338268efa687c1f424784a8d829723'
API_URL = "https://api.dmarket.com"

def generate_headers(method, api_path, body=None):
    nonce = str(round(datetime.now().timestamp()))
    string_to_sign = method + api_path

    if body is not None:
        string_to_sign += json.dumps(body)
    string_to_sign += nonce
    signature_prefix = "dmar ed25519 "
    encoded = string_to_sign.encode('utf-8')

    signature_bytes = crypto_sign(encoded, bytes.fromhex(secret_key))
    signature = signature_bytes[:64].hex()
    headers = {
        "X-Api-Key": public_key,
        "X-Request-Sign": signature_prefix + signature,
        "X-Sign-Date": nonce
    }
    return headers

params = {
    "gameId": "a8db",
    "title": " StatTrak™ Shadow Daggers | Freehand (Minimal Wear)",
    "currency": "USD",
    "limit": "100"
}


In [56]:
def get_items(params=None):
    method = 'GET'
    url_path = '/exchange/v1/market/items'
    if params:
        url_path += "?" + "&".join([f"{key}={value}" for key, value in params.items()])

    header = generate_headers(method, url_path)
    url = API_URL + url_path

    market_response = requests.get(url, headers=header)

    try:
        items = json.loads(market_response.text)
        return items
    except Exception as e:
        return e

In [57]:
class ItemTracker:
    def __init__(self):
        self._history_item_ids = []
        self._items = pd.DataFrame()  

    def get_items(self):
        return self._items

    def get_history_item_ids(self): 
        return self._history_item_ids

    def track_items(self, list_of_item_names: list, params: dict, price_offset: int = 50):
        max_price: int = params["priceTo"] 

        for item_name in list_of_item_names:
            go = True
            total_item_name_count = 0
            while go and params["priceTo"] <= max_price:
                params["title"] = item_name

                # Retrieve new items
                new_items = pd.json_normalize(get_items(params)["objects"])  

                if len(new_items) == 0:
                    print(f"No more results for title: {item_name}")
                    go = False

                else:
                    # Extend the history list with the new item ids and add the item info to the dataframe
                    self._history_item_ids.extend(new_items["extra.offerId"])
                    self._items = pd.concat([self._items, new_items], ignore_index=True)  

                    # Offset the prices 
                    current_top_price = self._items["price.USD"].astype(int).max()
                    params["priceFrom"] = current_top_price + 1
                    params["priceTo"] = current_top_price + 1 + price_offset
                    
                
                total_item_name_count += len(new_items) 

            print(f"Total count of {item_name}: {total_item_name_count}")

    def reset_items(self):
        self._items = pd.DataFrame()  
    
    def reset_history_item_ids(self):
        self._history_item_ids = []

In [58]:
titles = ["StatTrak™ P250 | Red Rock (Field-Tested)", "Souvenir P250 | Gunsmoke (Minimal Wear)", "Nova | Green Apple (Field-Tested)", "Snakebite Case"]

In [81]:
params = {
    "gameId": "a8db",
    "currency": "USD",
    "limit": 100,
    "priceFrom": 0,
    "priceTo": 400,
    "orderBy": "price",
    "orderDir": "asc"
}

In [82]:
titles = ["Snakebite Case"]

In [83]:
item_tracker = ItemTracker()
item_tracker.track_items(titles, params)

No more results for title: Snakebite Case
Total count of Snakebite Case: 2279


In [84]:
item_tracker.get_items()[item_tracker.get_items()["price.USD"]==32]

,itemId,type,amount,classId,gameId,gameType,inMarket,lockStatus,title,description,...,fees.dmarket.instantSell.default.minFee.USD,fees.dmarket.exchange.default.percentage,fees.dmarket.exchange.default.minFee.DMC,fees.dmarket.exchange.default.minFee.USD,discountPrice.DMC,discountPrice.USD,extra.viewAtSteam,extra.groupId,extra.tradeLock,extra.withdrawable


In [ ]:
item_tracker.ge

In [40]:
item_tracker.get_items()

,itemId,type,amount,classId,gameId,gameType,inMarket,lockStatus,title,description,...,fees.dmarket.sell.default.minFee.USD,fees.dmarket.instantSell.default.percentage,fees.dmarket.instantSell.default.minFee.DMC,fees.dmarket.instantSell.default.minFee.USD,fees.dmarket.exchange.default.percentage,fees.dmarket.exchange.default.minFee.DMC,fees.dmarket.exchange.default.minFee.USD,discountPrice.DMC,discountPrice.USD,extra.withdrawable
0,592b5fa4-c1da-5f4f-98a4-0e1ab823e5b2,offer,1,0:4418618853,a8db,steam,True,False,Snakebite Case,,...,,,,,,,,,,NaN
1,b7ff7ba9-a4e7-54e1-9870-82a5af47462f,offer,1,0:4418618853,a8db,steam,True,False,Snakebite Case,,...,,,,,,,,,,NaN
2,0:4418618853:31294195225:730,offer,1,0:4418618853,a8db,steam,False,False,Snakebite Case,,...,,,,,,,,,,NaN
3,fac4681c-189f-5b07-8019-72d66d2d40fc,offer,1,0:4418618853,a8db,steam,True,False,Snakebite Case,,...,,,,,,,,,,NaN
4,0:4418618853:26716750579:730,offer,1,0:4418618853,a8db,steam,False,False,Snakebite Case,,...,,,,,,,,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,35551e13-105c-557f-a6b8-4eb2f48088b3,offer,1,0:4418618853,a8db,steam,True,False,Snakebite Case,,...,,,,,,,,,,True
896,21b793c0-8062-5c3a-9b6f-2b8a82270a21,offer,1,0:4418618853,a8db,steam,True,False,Snakebite Case,,...,,,,,,,,,,True
897,49147749-968a-59e5-8cf8-74e4b5ceb017,offer,1,0:4418618853,a8db,steam,True,False,Snakebite Case,,...,,,,,,,,,,True
898,076378a0-8dd5-5f23-ac62-940d23bb7cd0,offer,1,0:4418618853,a8db,steam,True,False,Snakebite Case,,...,,,,,,,,,,True


In [38]:
data_frame = item_tracker.get_items()  # Assuming this returns a pandas DataFrame

for item in data_frame.itertuples(index=False):
    series_item = pd.Series(item, index=data_frame.columns)
    offer_id = series_item["extra.offerId"]
    print(offer_id)


NameError: name 'item_tracker' is not defined

In [45]:
item_tracker.get_items().columns


Index(['itemId', 'type', 'amount', 'classId', 'gameId', 'gameType', 'inMarket',
       'lockStatus', 'title', 'description', 'image', 'slug', 'owner',
       'ownersBlockchainId', 'status', 'discount', 'instantTargetId',
       'createdAt', 'productId', 'ownerDetails.id', 'ownerDetails.avatar',
       'ownerDetails.wallet', 'price.DMC', 'price.USD', 'instantPrice.DMC',
       'instantPrice.USD', 'exchangePrice.DMC', 'exchangePrice.USD',
       'suggestedPrice.DMC', 'suggestedPrice.USD',
       'recommendedPrice.offerPrice.DMC', 'recommendedPrice.offerPrice.USD',
       'recommendedPrice.d3.DMC', 'recommendedPrice.d3.USD',
       'recommendedPrice.d7.DMC', 'recommendedPrice.d7.USD',
       'recommendedPrice.d7Plus.DMC', 'recommendedPrice.d7Plus.USD',
       'extra.nameColor', 'extra.backgroundColor', 'extra.tradable',
       'extra.offerId', 'extra.isNew', 'extra.gameId', 'extra.name',
       'extra.categoryPath', 'extra.linkId', 'extra.quality', 'extra.category',
       'extra.tradeLoc

In [53]:
items = item_tracker.get_items()

for item in items.itertuples(index=False):
    series_item = pd.Series(item, index=items.columns)
    print(series_item["extra.tradeLock"])

nan
nan
nan
nan
7.0
7.0
7.0
7.0
5.0
6.0
5.0
5.0
5.0
6.0
5.0
5.0
6.0
5.0
5.0
6.0
5.0
6.0
5.0
5.0
6.0
6.0
6.0
5.0
5.0
5.0
5.0
6.0
5.0
5.0
5.0
6.0
5.0
5.0
5.0
5.0
7.0
6.0
5.0
6.0
5.0
2.0
5.0
6.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
7.0
6.0
5.0
5.0
5.0
6.0
5.0
7.0
6.0
6.0
6.0
nan
5.0
5.0
5.0
5.0
6.0
5.0
5.0
6.0
6.0
6.0
7.0
5.0
5.0
5.0
5.0
5.0
6.0
5.0
5.0
6.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
6.0
7.0
6.0
7.0
7.0
7.0
7.0
7.0
7.0
6.0
7.0
6.0
6.0
7.0
7.0
6.0
6.0
6.0
7.0
7.0
6.0
6.0
7.0
7.0
6.0
6.0
6.0
6.0
7.0
6.0
7.0
6.0
6.0
7.0
7.0
6.0
6.0
6.0
6.0
6.0
7.0
6.0
nan
7.0
nan
nan
7.0
6.0
5.0
nan
nan
5.0
7.0
7.0
nan
nan
nan
nan
nan
4.0
nan
nan
7.0
nan
6.0
nan
nan
nan
nan
6.0
nan
nan
nan
6.0
nan
nan
2.0
nan
nan
nan
nan
nan
nan
4.0
5.0
nan
7.0
6.0
nan
nan
nan
6.0
nan
nan
nan
6.0
6.0
nan
nan
6.0
5.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [51]:
items = item_tracker.get_items()
items

,itemId,type,amount,classId,gameId,gameType,inMarket,lockStatus,title,description,...,fees.dmarket.instantSell.default.minFee.USD,fees.dmarket.exchange.default.percentage,fees.dmarket.exchange.default.minFee.DMC,fees.dmarket.exchange.default.minFee.USD,discountPrice.DMC,discountPrice.USD,extra.viewAtSteam,extra.groupId,extra.withdrawable,extra.tradeLock
0,0:4418618853:30287006109:730,offer,1,0:4418618853,a8db,steam,False,False,Snakebite Case,,...,,,,,,,NaN,NaN,NaN,NaN
1,6a3a6293-0374-5b4c-bdf6-4c1fa0b2a4c6,offer,1,0:4418618853,a8db,steam,True,False,Snakebite Case,,...,,,,,,,https://steamcommunity.com/profiles/7656119927...,76561199270992789,True,NaN
2,0:4418618853:25981156451:730,offer,1,0:4418618853,a8db,steam,False,False,Snakebite Case,,...,,,,,,,NaN,NaN,NaN,NaN
3,c8cfa35e-220b-521f-b4e3-b8e5918edffc,offer,1,0:4418618853,a8db,steam,True,False,Snakebite Case,,...,,,,,,,https://steamcommunity.com/profiles/7656119929...,76561199290458753,True,NaN
4,67064cc6-985a-5ff1-8dee-9192b4d0fe10,offer,1,0:4418618853,a8db,steam,True,False,Snakebite Case,,...,,,,,,,https://steamcommunity.com/profiles/7656119929...,76561199296507101,NaN,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,183bc406-dcf7-5baf-a03f-1ba5e0a7d06c,offer,1,0:4418618853,a8db,steam,True,False,Snakebite Case,,...,,,,,,,https://steamcommunity.com/profiles/7656119948...,76561199489232927,True,NaN
996,7d660ed1-2e73-5e28-a2d1-07aea9c0ab1f,offer,1,0:4418618853,a8db,steam,True,False,Snakebite Case,,...,,,,,,,https://steamcommunity.com/profiles/7656119929...,76561199295504372,True,NaN
997,15df4a96-b4f3-56ec-a78e-d559fa7f9f99,offer,1,0:4418618853,a8db,steam,True,False,Snakebite Case,,...,,,,,,,https://steamcommunity.com/profiles/7656119926...,76561199266540755,True,NaN
998,2e3da931-dbd0-5cf5-beea-50073d85aac7,offer,1,0:4418618853,a8db,steam,True,False,Snakebite Case,,...,,,,,,,https://steamcommunity.com/profiles/7656119937...,76561199378922483,True,NaN


In [8]:
params = {
    "gameId": "a8db",
    "currency": "USD",
    "limit": 100,
    "priceFrom": 0,
    "priceTo": 100,
    "orderBy": "price",
    "orderDir": "asc",
}

In [9]:
item_tracker = ItemTracker()
item_tracker.track_items(["Snakebite Case"], params)

Total count of Snakebite Case: 1100


In [22]:
for i in items["deliveryStats.time"]:
    print(i)

0
0
























































































































































































































































































































0























0























































































0











0






































































































































































































































0






































































































































0



11946



11946



11946





11946












11946











11946
11946










11946
11946














11946








0
11946




































0














In [13]:
items = item_tracker.get_items()

0       29
1       29
2       31
3       31
4       31
        ..
1095    51
1096    51
1097    51
1098    51
1099    51
Name: price.USD, Length: 1100, dtype: object

In [11]:
item_tracker.get_items()["extra.tradeLock"]

0       NaN
1       NaN
2       7.0
3       7.0
4       7.0
       ... 
1095    NaN
1096    NaN
1097    NaN
1098    NaN
1099    NaN
Name: extra.tradeLock, Length: 1100, dtype: float64

In [12]:
items

NameError: name 'items' is not defined

In [100]:
a = dict((k, v) for v, k in enumerate(titles))
a

{'StatTrak™ P250 | Red Rock (Field-Tested)': 0,
 'Souvenir P250 | Gunsmoke (Minimal Wear)': 1,
 'Nova | Green Apple (Field-Tested)': 2,
 'Snakebite Case': 3}

In [99]:
for i in titles:
    while 

StatTrak™ P250 | Red Rock (Field-Tested)
Souvenir P250 | Gunsmoke (Minimal Wear)
Nova | Green Apple (Field-Tested)
Snakebite Case


new_items

In [69]:
params = {
    "gameId": "a8db",
    "currency": "USD",
    "limit": 100,
    "priceFrom": 0,
    "priceTo": 100,
    "orderBy": "price",
    "orderDir": "asc",
}

unique_item_ids = set()

while True:
    response = get_items(params)
    items = response.get("objects", [])  # Check if "objects" key exists in the response
    if not items:
        break
    
    # Collect unique item IDs
    item_ids = [item["extra"]["offerId"] for item in items]
    unique_item_ids.update(item_ids)
    
    print(f"{len(items)} items, {len(unique_item_ids)} unique item IDs")
    
    # Check if there are more items to retrieve
    if len(items) < params["limit"]:
        break
    
    # Find the maximum offerId from the current batch of items
    max_offer_id = max(item_ids)
    
    # Update the params to retrieve the next batch of items
    params["offset"] = max_offer_id
    params["limit"] = 100  # You can adjust the limit if needed
    
print(f"Total unique item IDs: {len(unique_item_ids)}")


100 items, 100 unique item IDs
Total unique item IDs: 100


In [119]:
import datetime

def test_created_at():
    now = datetime.datetime.now()
    five_minutes_ago = now - datetime.timedelta(minutes=5)
    
    # Convert datetime objects to timestamps
    now_timestamp = now.timestamp()
    five_minutes_ago_timestamp = five_minutes_ago.timestamp()
    
    # Calculate the average timestamp
    avg_timestamp = (now_timestamp + five_minutes_ago_timestamp) / 2
    
    # Convert the average timestamp back to a datetime object
    created_at = datetime.datetime.fromtimestamp(avg_timestamp)
    
    print(f"Current time: {now}")
    print(f"Time 5 minutes ago: {five_minutes_ago}")
    print(f"Calculated created_at: {created_at}")

test_created_at()


Current time: 2023-07-01 20:34:41.215982
Time 5 minutes ago: 2023-07-01 20:29:41.215982
Calculated created_at: 2023-07-01 20:32:11.215982


In [7]:
a = None
b = None

if not a and not b:
    print("a")

a


In [16]:
a = None
if a:
    print('a')

In [85]:
def extract_sales_history(encoded_item):
    market_response = requests.get(f'https://api.dmarket.com/marketplace-api/v1/last-sales?Title={encoded_item}&GameID=a8db&Currency=USD')
    
    print(market_response.text)
    sales = json.loads(market_response.text)['LastSales']

    return sales

In [86]:
import statistics
from urllib import parse

In [87]:
def encode_item(item_name):
    new_string = parse.quote(item_name)

    return new_string

In [88]:
def calculate_seconds(date_from, date_to):
   
    time_difference = date_to - date_from 
    days_sale = time_difference.days * 86400
    second_sale = time_difference.seconds
    time_difference_seconds = days_sale+ second_sale
 
    return round((time_difference_seconds / 3600) , 2)

In [89]:
def extract_sales_information(sales_info):
    if sales_info != None:
        return_obj = {'total_count': len(sales_info)}

        Date = []
        Price = []
        last_record = None
        time_diff = []
        last_20_sales = []
        for sale in sales_info:

            Date.append(int(sale['Date']))
            Price.append(int(sale['Price']['Amount']))

            if len(last_20_sales) < 20:
                last_20_sales.append(int(sale['Price']['Amount'])/100)
            
            if last_record == None :
                last_record = datetime.fromtimestamp(int(sale['Date']))
               
            else: 
                new_record = datetime.fromtimestamp(int(sale['Date']))
                time_diff.append(calculate_seconds(new_record ,last_record))
                last_record = datetime.fromtimestamp(int(sale['Date']))
        try:
            return_obj['min_price'] = min(Price) /100
        except:
            return_obj['min_price'] = 0
        try:
            return_obj['max_price'] = max(Price)/100
        except :
            return_obj['max_price'] = 0 
        return_obj['mean_price'] = statistics.mean(Price) / 100
        return_obj['mode_price'] = statistics.mode(Price)/ 100
        return_obj['last_sale'] = calculate_seconds( datetime.fromtimestamp(int(sales_info[0]['Date'])), datetime.now())
        return_obj['avg_sale_time'] = statistics.mean(time_diff) 
        return_obj['last_10_sales'] = last_20_sales

        return return_obj
    return None

In [91]:
sales_history = extract_sales_history("Snakebite Case")


{"LastSales":[{"Date":"1688570722","Price":{"Currency":"USD","Amount":"31"}},{"Date":"1688570722","Price":{"Currency":"USD","Amount":"31"}},{"Date":"1688570722","Price":{"Currency":"USD","Amount":"31"}},{"Date":"1688570722","Price":{"Currency":"USD","Amount":"31"}},{"Date":"1688570572","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688570484","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688570484","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688570144","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688570104","Price":{"Currency":"USD","Amount":"32"}},{"Date":"1688569849","Price":{"Currency":"USD","Amount":"32"}},{"Date":"1688569849","Price":{"Currency":"USD","Amount":"32"}},{"Date":"1688569849","Price":{"Currency":"USD","Amount":"32"}},{"Date":"1688569101","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688568351","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688568308","Price":{"Currency":"USD","Amount":"32"}},{"Date":"1688568308","Price":{"Currency":

In [95]:
datetime.fromtimestamp(1688570722)

datetime.datetime(2023, 7, 5, 17, 25, 22)

In [92]:
sales_history

[{'Date': '1688570722', 'Price': {'Currency': 'USD', 'Amount': '31'}},
 {'Date': '1688570722', 'Price': {'Currency': 'USD', 'Amount': '31'}},
 {'Date': '1688570722', 'Price': {'Currency': 'USD', 'Amount': '31'}},
 {'Date': '1688570722', 'Price': {'Currency': 'USD', 'Amount': '31'}},
 {'Date': '1688570572', 'Price': {'Currency': 'USD', 'Amount': '29'}},
 {'Date': '1688570484', 'Price': {'Currency': 'USD', 'Amount': '29'}},
 {'Date': '1688570484', 'Price': {'Currency': 'USD', 'Amount': '29'}},
 {'Date': '1688570144', 'Price': {'Currency': 'USD', 'Amount': '29'}},
 {'Date': '1688570104', 'Price': {'Currency': 'USD', 'Amount': '32'}},
 {'Date': '1688569849', 'Price': {'Currency': 'USD', 'Amount': '32'}},
 {'Date': '1688569849', 'Price': {'Currency': 'USD', 'Amount': '32'}},
 {'Date': '1688569849', 'Price': {'Currency': 'USD', 'Amount': '32'}},
 {'Date': '1688569101', 'Price': {'Currency': 'USD', 'Amount': '29'}},
 {'Date': '1688568351', 'Price': {'Currency': 'USD', 'Amount': '29'}},
 {'Dat

In [31]:
encoded_item = encode_item("Snakebite Case")
sales_history = extract_sales_history("Snakebite Case")

{"LastSales":[{"Date":"1688497487","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688497338","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688497192","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688496944","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688496576","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688496444","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688496444","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"46"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"49"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}},{"Date":"1688496165","Price":{"Currency":

In [13]:
import datetime

def check_appeared_in_sales_history(sales_history, interval_minutes):
    current_time = datetime.datetime.now()
    interval_start = current_time - datetime.timedelta(minutes=interval_minutes)
    
    for sale in sales_history:
        sale_time = datetime.datetime.fromtimestamp(int(sale['Date']))
        if interval_start <= sale_time <= current_time:
            return True
    
    return False


In [24]:
datetime.datetime.fromtimestamp(int(1688489977))
# current_time = datetime.datetime.now()
# current_time

check_appeared_in_sales_history(sales_history, 20)

True

In [25]:
datetime.datetime.fromtimestamp(int(1688489977))

datetime.datetime(2023, 7, 4, 18, 59, 37)

In [ ]:
if check_appeared_in_sales_history(sales_history, 5) and 

In [29]:
a = [1, 2, 3, 4]

In [28]:
b = [1,2,3,4,5]

In [30]:
import pandas as pd

def collect_items(item_tracker, current_ids, data_writer, table_name, original_offer_ids, list_of_item_names, params, periodicity):
    # Make a copy of parameters
    params_copy = params.copy()

    # Retrieve updated items
    item_tracker.track_items(list_of_item_names, params_copy)
    new_items = item_tracker.get_items()

    # Check for disappeared items
    disappeared_ids = [offer_id for offer_id in current_ids if offer_id not in new_items["extra.offerId"].values]

    # Update the current_ids with new items
    current_ids.extend(new_items["extra.offerId"])

    # Iterate over new items and insert into the database if offer ID is unique
    for item in new_items.itertuples(index=False):
        series_item = pd.Series(item, index=new_items.columns)
        offer_id = series_item["extra.offerId"]

        # If the item is NOT unique, break
        if offer_id in original_offer_ids:
            break

        else:
            series_item = series_item.replace('', None)

            # Collect item data
            data = collect_item_data(series_item, periodicity)
            # Save data to database
            data_writer.write_item_data(data, table_name)

    return disappeared_ids


# Test with basic lists
class ItemTracker:
    def __init__(self):
        self.items = pd.DataFrame({"extra.offerId": [1, 2, 3]})  # Mock items data

    def track_items(self, list_of_item_names, params):
        pass

    def get_items(self):
        return self.items


class DataWriter:
    def write_item_data(self, data, table_name):
        pass


def collect_item_data(series_item, periodicity):
    pass


def test_collect_items():
    # Mock data
    item_tracker = ItemTracker()
    current_ids = [1, 2, 3]
    data_writer = DataWriter()
    table_name = "items"
    original_offer_ids = [1, 2, 3]
    list_of_item_names = ["Item1", "Item2", "Item3"]
    params = {}
    periodicity = 5

    # Call the function
    disappeared_ids = collect_items(item_tracker, current_ids, data_writer, table_name, original_offer_ids, list_of_item_names, params, periodicity)

    # Check the result
    assert disappeared_ids == [], "Disappeared IDs are not as expected"

    print("Test passed successfully.")


test_collect_items()


Test passed successfully.


In [41]:
a = []

In [42]:
#How to check if a list is not empty ?

True

In [43]:
import datetime

In [45]:
a ={"LastSales":
 [{"Date":"1688497487","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688497338","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688497192","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688496944","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688496576","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688496444","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688496444","Price":{"Currency":"USD","Amount":"29"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"46"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"49"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"46"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}},{"Date":"1688496165","Price":{"Currency":"USD","Amount":"48"}}]}


In [54]:
a["LastSales"][0]

{'Date': '1688497487', 'Price': {'Currency': 'USD', 'Amount': '29'}}

In [97]:
for i in range(3):
    print(i)

0
1
2


In [40]:
a = [str(i) for i in range(3)]

In [41]:
def lol(a):
    # Create a copy of the list
    a_copy = a.copy()
    
    # Iterate over the copy and remove elements from the original list
    for i in a_copy:
        a.remove(i)

In [42]:
for i in a:
    print(i)

0
1
2


In [43]:
lol(a)

In [44]:
a

[]